In [ ]:
import pandas as pd
import numpy as np
import collections
import datetime
import re
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.linear_model import LogisticRegressionCV
from functools import reduce
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import NearMiss
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, RocCurveDisplay, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, PrecisionRecallDisplay 
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFECV
from sklearn.model_selection import learning_curve

In [ ]:
Helper Functions: plot_lift()

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df = pd.read_csv(r"HIS_combined_821.csv",index_col = 0).reset_index(drop=True)

In [ ]:
idx_to_remove = df_removed[df_removed['消费项目'].str.contains('口腔常规检查', regex=False)].reset_index()['index']
#idx_to_remove.head()
#print(idx_to_remove.shape, len(o))
df_leftjoined2=df_removed.reset_index().merge(idx_to_remove, on='index', how='left', indicator=True)
#df_leftjoined2.head()
df_removed2 = df_leftjoined2[~(df_leftjoined2['_merge']=='both')].drop('_merge', axis=1)
df_removed2.shape

In [ ]:
idx_to_remove = df_removed[df_removed['收费分类'].str.contains('种植费', regex=False) & df_removed['消费项目'].str.contains('手术一次性材料费（手术时收）', regex=False)]\
                         .reset_index()['index']
df_leftjoined3=df_removed2.merge(idx_to_remove, on='index', how='left', indicator=True)

df_removed3 = df_leftjoined3[~(df_leftjoined3['_merge']=='both')].drop('_merge', axis=1)
df_removed3.shape

## 2. processing positive and negative patient dataframe
### 2.1 positive dataset

In [ ]:
# series of patient_uid with implant billing 
pt_uid_implant = df_removed3[df_removed3['收费分类']=="种植费"]['patient_uid'].drop_duplicates()

# create subset of df pt_implant_all for patient_uid with implant billing 
pt_implant_all=df_removed3.merge(pt_uid_implant, on='patient_uid', how='inner')

pt_implant_all['date']=pd.to_datetime(pt_implant_all['消费时间']).dt.date
pt_implant_all = pt_implant_all.fillna('Unknown')

print(df_removed3.shape, pt_implant_all.shape)#,  pat_implant1.shape)
pt_implant_all.head()

In [ ]:
# from the implant df subset, select the earliest date with implant billing for every patient as the implant date
pt_implant_all.sort_values(by=['patient_uid', 'date'], ascending=True, inplace=True)
pt_implant_date = pt_implant_all[pt_implant_all['收费分类']=="种植费"][['patient_uid', 'date']]\
                                .groupby('patient_uid').agg(implant_date=('date', min))


# create a column 'implant date' and 'ayearbfore_implant_date' for every patient 
pt_implant_all1=pt_implant_all.set_index('patient_uid')
for pt in pt_uid_implant:
    pt_implant_all1.loc[pt, 'implant_date']=pt_implant_date.loc[pt, 'implant_date']
    
pt_implant_all1['a_year_bfore_implant_date'] = pt_implant_all1['implant_date']-datetime.timedelta(days= 365) 


# filter pt records within specified time window, 
# that is, dates within a year prior to the implant date and implant date is at least a year before the start date 2018/01/01 
pt_implant_all2 = pt_implant_all1[(pt_implant_all1['date'] < pt_implant_all1['implant_date'])\
                & (pt_implant_all1['date'] >= pt_implant_all1['a_year_bfore_implant_date'])\
                & (pt_implant_all1['a_year_bfore_implant_date'] >= pd.to_datetime('2018-01-01 00:00')) ].drop('index', axis=1)

print(pt_implant_all1.shape, pt_implant_date, pt_implant_all2.shape)
pt_implant_all2.head()

In [ ]:
print('unique pt: before time window "{}", after "{}"\n------------------------'.format(pt_uid_implant.shape[0], len(pt_implant_all2.index.unique())))

In [ ]:
def create_feature_df(df, groupby_field='patient_uid'):

    f1 = df.groupby('patient_uid').agg(diagnosis=('诊断名称', 'unique'))
    f2 = df.groupby('patient_uid').agg(treatment_plan=('治疗计划', 'unique'))
    f3 = df.groupby('patient_uid').agg(purchase=('消费项目', 'unique'))
    f4 = df.groupby('patient_uid').agg(total_order=('项目金额（实付）', 'sum'))
    f5 = df.groupby('patient_uid').agg(dept=('科室', 'unique'))
    f6 = df.groupby('patient_uid').agg(gender=('性别', 'unique'))
    f7 = df.groupby('patient_uid').agg(age=('年龄', 'max'))
    f8 = df.groupby('patient_uid').agg(dxcode=('诊断代码', 'unique'))
    f9 = df.groupby('patient_uid').agg(order_type=('收费分类', 'unique'))

    f10 = df.groupby(['patient_uid', 'date']).agg(order_by_visit=('项目金额（实付）', 'sum'))\
                        .groupby('patient_uid').agg(mean_order=('order_by_visit', 'mean'))

    f11 = df.groupby('patient_uid').agg(date_list=('date', 'unique'))
    f11['duration_days'] = f11['date_list'].map(lambda x: (x.max()-x.min()).days)
    
    f12 = df.groupby('patient_uid').agg(visits=('date', 'nunique'))
    
    
    f_list = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12]
    linked_df = reduce(lambda x, y: x.merge(y, on='patient_uid', how='inner'), f_list)
    linked_df['visit_interval']=linked_df['duration_days']/linked_df['visits']
    return linked_df

linked_implant_df = create_feature_df(pt_implant_all2)
linked_implant_df.head()

### 2.2. Negative dataset

In [ ]:
df_removed4 = df_removed3.merge(pt_uid_implant, on='patient_uid', how='left', indicator=True)

pt_noimplant_all = df_removed4[df_removed4['_merge']=='left_only'].drop('_merge', axis=1)

pt_noimplant_all['date']=pd.to_datetime(pt_noimplant_all['消费时间']).dt.date
pt_noimplant_all = pt_noimplant_all.fillna('Unknown')

pt_noimplant_all_1=pt_noimplant_all.set_index('patient_uid')
pt_uid_noimplant = pt_noimplant_all_1.index.unique()

In [ ]:
# from the noimplant pt subset, select the latest date for every patient, apply a time windeow to filter records within a year from the latest date
pt_noimplant_lastest_date = pt_noimplant_all.groupby('patient_uid').agg(latest_date=('date', 'max'))

# create a column 'implant date' and 'ayearbfore_implant_date' for every patient 
for pt in pt_uid_noimplant:
    pt_noimplant_all_1.loc[pt, 'latest_date']=pt_noimplant_lastest_date.loc[pt, 'latest_date']
    
pt_noimplant_all_1['a_year_bfore_latest_date'] = pt_noimplant_all_1['latest_date']-datetime.timedelta(days= 365) 
# pt_noimplant_all_1.head()

pt_noimplant_all_2=pt_noimplant_all_1[ (pt_noimplant_all_1['date'] > pt_noimplant_all_1['a_year_bfore_latest_date'])\
                   & (pt_noimplant_all_1['date'] < pt_noimplant_all_1['latest_date'])
                   & (pt_noimplant_all_1['a_year_bfore_latest_date'] > pd.to_datetime('2018-01-01 00:00')) ].drop('index', axis=1).drop_duplicates()

#print(pt_noimplant_all_1.shape, pt_noimplant_lastest_date, pt_noimplant_all_2.shape)
#print(pre_noimplant.shape, pt_noimplant_all_2.shape)

In [ ]:
linked_noimplant_df = create_feature_df(pt_noimplant_all_2)
linked_noimplant_df.head()

In [ ]:
print('Dataframe of implant patient from YH {}'.format(linked_implant_df.shape))#, link_implant.shape))
print('Dataframe of non-implant patient from YH {}'.format(linked_noimplant_df.shape))#, link_noimplant.shape))

In [ ]:
linked_noimplant_df['implant']=0
linked_implant_df['implant']=1

linked_all = linked_noimplant_df.append(linked_implant_df)
linked_all.shape

## 3. EDA, Data Cleaning, Built Features

In [ ]:
linked_all.info()

In [ ]:
linked_all[linked_all['implant']==1].describe()
linked_all[linked_all['implant']==0].describe()

In [ ]:
# plt.boxplot([linked_all[linked_all['implant']==1]['total_order'], linked_all[linked_all['implant']==0]['total_order']]);

# ax = linked_all[linked_all['implant']==0]['total_order'].plot.hist(density=True)

def plt_compare_hist(df, label, col_name, bins=50, density=True):
    
    _, bins, _ = plt.hist(df[df[label]==0][col_name], bins=bins, density=density, label='negative')#, normed=True)
    _ = plt.hist(linked_all[linked_all[label]==1][col_name], bins=bins, alpha=0.5, density=True, label='positive')#, normed=True)
    plt.legend(loc='upper right')
    plt.show();
    return

def create_compare_violin(df, col_name, label='implant'):
    sns.violinplot(data=df, x=label, y=col_name, hue=label, palette="muted", split=True)
    return

### 3.1. EDA, remove outlier in "total_order"

In [ ]:
plt_compare_hist(linked_all, 'implant', 'total_order')

In [ ]:
#linked_all[(linked_all['implant']==0) & (linked_all['total_order']>50000)].head()
#linked_all[(linked_all['implant']==1) & (linked_all['total_order']>50000)].head()
linked_all1=linked_all[~(linked_all['total_order']>40000)]
print(linked_all.shape, linked_all1.shape)
plt_compare_hist(linked_all1, 'implant', 'total_order')

### 3.2. EDA of mean_order

In [ ]:
plt_compare_hist(linked_all1, 'implant', 'mean_order')

### 3.3. EDA, Cleaning of "age"

In [ ]:
linked_all2=linked_all1[~(linked_all1['age']<10)]
plt_compare_hist(linked_all2, 'implant', 'age')

In [ ]:
plt_compare_hist(linked_all2, 'implant', 'duration_days')

In [ ]:
plt_compare_hist(linked_all2, 'implant', 'visits', 10)

In [ ]:
plt_compare_hist(linked_all2, 'implant', 'visit_interval', 10)

### 3.4. Cleaning, Contingency Table for "gender"

In [ ]:
# Remove pt with mulitple gender
linked_all3 = linked_all2[~(linked_all2['gender'].map(lambda x: len(x)) > 1)].copy()

linked_all3.loc[:, 'gender'] = linked_all3.loc[:,'gender'].map(lambda x: x[0]).map({'女': 0, '男': 1})
linked_all3['gender']

In [ ]:
#temp = linked_all3.reset_index().groupby(['implant', 'gender']).agg(cnt=('patient_uid', 'count'))

crosstab = linked_all3.reset_index().pivot_table(index='gender', columns='implant', values='patient_uid', aggfunc='count', margins=True)
crosstab
crosstab1=crosstab.copy()
crosstab1.loc[:,0:1]=crosstab1.loc[:,0:1].div(crosstab1['All'], axis=0)
crosstab1

### 3.5. Cleaning of "diagnosis" and creating a patient diagnosis sparse matrix, truncate low frequency diagnosis < 10

In [ ]:
def cleaned_text(d: list):
    return [o for o in re.sub(r'[^\u4e00-\u9fff]', ' ', str(d)).split(" ") if (o != "") & (len(o) > 0) ]

def cleaned_code(d: list):
    return [x for x in  re.findall(r'[a-zA-Z]\d{2}.\d{3}', str(d))]

def create_pt_categorical_feature_matrix(df0, col, truncated_n=0):
    
    df=df0.copy()
    # clean text in col
    l = []
    for d in df[col]:    
        l += cleaned_text(d)
    
    # create a corpus of categorical features from col
    bag_of_cat_feature = set(l)
    
    df.loc[:, col] = df.loc[:, col].map(cleaned_text)
    
    # create a dict of {feature: count of feature}
    cnt_feature=dict()
    for d in bag_of_cat_feature:
        v = 0
        for m in l:
            if m==d:
                v+=1
        cnt_feature[d]=v
    
    if truncated_n>0:
        cnt_feature_sorted = {k: v for k,v in sorted(cnt_feature.items(), key=lambda item: item[1], reverse=True) if v>truncated_n}
    else:
        cnt_feature_sorted = {k: v for k,v in sorted(cnt_feature.items(), key=lambda item: item[1], reverse=True)}
    
    # iterate on pt_uid and col to create  {pt_uid: one-hot coding of categorical feature dict}
    bag_of_featuer_sorted = set([k for k in cnt_feature_sorted.keys()])
    
    # create a {patient_uid: feature coding dict}
    pt_feature_dict=dict()  
    for pt in df.index:
        pt_feature=df.loc[pt, col]

        # Iterating bag_of_featuer_sorted, if feature in "col" = 1, else = 0 
        feature_matrix = dict()
        for d in list(bag_of_featuer_sorted):
            if d in pt_feature:
                feature_matrix[d]=1
            else:
                feature_matrix[d]=0

        pt_feature_dict[pt]=feature_matrix
    
    # create a pt_uid, categorical feature dataframe and rename columns
    df_pt_feature = pd.DataFrame.from_dict(pt_feature_dict, orient='index')
    
    cols_feature_dict = {v: col+str(i) for i, v in enumerate(cnt_feature_sorted)}

    df_pt_feature.rename(columns=cols_feature_dict, inplace=True)
    
    return df_pt_feature, cols_feature_dict, cnt_feature_sorted


def create_pt_vector(df0, col, vector_type, truncated_n, is_dxcode=False):
    
    df=df0.copy()
    # clean text in col
    l = []
    
    if is_dxcode:
        for d in df[col]:    
            l += cleaned_code(d)
        bag_of_cat_feature = set(l)
        df.loc[:, col] = df.loc[:, col].map(cleaned_code).astype(str)
    
    else:
        for d in df[col]:    
            l += cleaned_text(d)
    # create a corpus of categorical features from col
        bag_of_cat_feature = set(l)
        df.loc[:, col] = df.loc[:, col].map(cleaned_text).astype(str)
    
    
    if vector_type == 'tf':
        # using count-Vector 
        if is_dxcode:
            token_pattern_dxcode=r'[a-zA-Z]\d{2}.\d{3}'
            vectorizer = CountVectorizer(min_df=truncated_n, token_pattern=token_pattern_dxcode)
        else:
            vectorizer = CountVectorizer(min_df=truncated_n)  
        X = vectorizer.fit_transform(df[col])
        bag_of_features = vectorizer.get_feature_names() 
        count_of_features = X.toarray().sum(axis=0)
        
        # using tf-idf vectorizer 
    elif vector_type =='tfidf':
        if is_dxcode:
            token_pattern_dxcode=r'[a-zA-Z]\d{2}.\d{3}'
            vectorizer = TfidfVectorizer(min_df=truncated_n, token_pattern=token_pattern_dxcode)
        else:
            vectorizer = TfidfVectorizer(min_df=truncated_n)
            
        X = vectorizer.fit_transform(df[col])
        bag_of_features = vectorizer.get_feature_names() 
        count_of_features = vectorizer.idf_
        
    cnt_feature = dict(zip(bag_of_features, count_of_features))
    feature_vector_sorted = {k: v for k,v in sorted(cnt_feature.items(), key=lambda item: item[1], reverse=True)}

    cols_feature_map = {v: col+str(i) for i, v in enumerate(feature_vector_sorted)}
    feature_cols_map = {v:k for k, v in cols_feature_map.items() }
    df1 = pd.DataFrame(X.toarray(), columns=bag_of_features, index=df0.index)

    out_df = df1.rename(columns=cols_feature_map)
        
    return out_df, feature_vector_sorted, feature_cols_map


### 3.6. Create a "dx" feature matrix

In [ ]:
df_pt_dx, dx_vector_dict, dx_col_map= create_pt_vector(linked_all3, 'diagnosis', 'tfidf', 10)
len(dx_vector_dict)

In [ ]:
df_pt_dx['implant']=linked_all3['implant']
corr = df_pt_dx.corr()

plt.figure(figsize=(12,10))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, vmax=1.0, vmin=-1.0, cmap=cmap)
plt.show();

In [ ]:
drop_diagnosis = corr[corr['implant'].abs()<0.005].index
{d:dx_col_map[d] for d in drop_diagnosis}

In [ ]:
implant_pt_dx = df_pt_dx[df_pt_dx['implant']==1].sum().drop('implant').to_dict()
temp_implant = pd.DataFrame().from_dict(implant_pt_dx, orient='index', columns=['implant_pt_cnt']).sort_values(by=['implant_pt_cnt'], ascending=False)
temp_implant.head(10)

noimplant_pt_dx = df_pt_dx[df_pt_dx['implant']==0].sum().to_dict()
temp_noimplant = pd.DataFrame.from_dict(noimplant_pt_dx, orient='index', columns=['noimplant_pt_cnt']).sort_values(by=['noimplant_pt_cnt'], ascending=False)
temp_noimplant.head(10)

In [ ]:
def pca_feature_reduction(df, cols, pca_alias, label_col, n_components=2):
    
    x = df.loc[:, cols].values
    x = StandardScaler().fit_transform(x)
    
    y = df.loc[:, label_col].values
    
    pca = PCA(n_components=n_components)
    pca_components = pca.fit_transform(x)
    print('cumulative varation: {}\nExplained variation per principal component: {}'.format(pca.explained_variance_ratio_.sum(), pca.explained_variance_ratio_))
    
    xi = np.arange(1, n_components+1, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)
    fig = plt.figure(figsize = (8,8))
    plt.plot(xi, y, marker='o', linestyle='--', color='b')
    
    pca_cols = [pca_alias+'_pca_'+str(k) for k in range(n_components)]
    pca_df = pd.DataFrame(data=pca_components, columns=pca_cols, index=df.index)
    
    #final_pca_df=pca_df.merge(df[[label_col]].reset_index(), on='index', how='left').set_index('index')
    final_pca_df=pd.concat([pca_df, df[[label_col]]], axis=1)#.merge(df[[label_col]].reset_index(), on='index', how='left').set_index('index')
    
    # scatter plot of 2-D pca
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    targets = [0, 1]
    colors = ['g', 'r']
    for target, color in zip(targets, colors):
        indicesToKeep = final_pca_df[label_col] == target
        ax.scatter(final_pca_df.loc[indicesToKeep, pca_alias+'_pca_0'], final_pca_df.loc[indicesToKeep, pca_alias+'_pca_1'], c=color, s=20, alpha=0.3);
    ax.legend(targets);
    #ax.grid();
    
    return final_pca_df#, pca_components.explained_variance_ratio_

from sklearn.manifold import TSNE

def tsne_marker(df, cols, alias, label_col, n=2):
    
    x = df.loc[:, cols].values
    x = StandardScaler().fit_transform(x)
    
    y = df.loc[:, label_col].values
   

    #pca = PCA(n_components=m)
    tsne = TSNE(n_components=n)#, perplexity=20, n_iter=1000, learning_rate=200)
    #tsne_after_pca = Pipeline([('pca', pca), ('tsne', tsne)])
    dr = tsne.fit_transform(x)
   
    tsne_cols = [alias+'_tsne_'+str(k) for k in range(n_components)]
    tsne_df = pd.DataFrame(data=dr, columns=tsne_cols, index=df.index)
    final_tsne_df=pd.concat([tsne_df, df[[label_col]]], axis=1)
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    targets = [0, 1]
    colors = ['g', 'r']
    for target, color in zip(targets, colors):
        indicesToKeep = final_tsne_df[label_col] == target
        ax.scatter(final_tsne_df.loc[indicesToKeep, alias+'_tsne_0'], final_tsne_df.loc[indicesToKeep, alias+'_tsne_1'], c=color, s=20, alpha=0.3);
    ax.legend(targets);
    
    return tsne_df

In [ ]:
cols = [c for c in df_pt_dx.columns if 'diagnosis' in c]
df_pt_dx['implant']=linked_all3['implant']

dx_pca_df = pca_feature_reduction(df_pt_dx, cols, 'dx', 'implant', 20)
dx_pca_df.head()

### 3.7. Create a "dxcode" feature matrix

In [ ]:
is_code=True
df_pt_dxcode, dxcode_vector_dict, dxcode_col_map= create_pt_vector(linked_all3, 'dxcode', 'tfidf', 10, is_code)
len(dxcode_vector_dict)
dxcode_vector_dict

In [ ]:
df_pt_dxcode['implant']=linked_all3['implant']
corr = df_pt_dxcode.corr()

plt.figure(figsize=(12,10))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, vmax=1.0, vmin=-1.0, cmap=cmap)
plt.show();

In [ ]:
corr['implant']

In [ ]:
drop_dxcode = corr[corr['implant'].abs()<0.005].index
{d: dxcode_col_map[d] for d in drop_dxcode}
df_pt_dxcode.shape

In [ ]:
df_pt_dxcode.drop(drop_dxcode, axis=1, inplace=True)
df_pt_dxcode.shape

In [ ]:
cols = [c for c in df_pt_dxcode.columns if 'dxcode' in c]
df_pt_dxcode['implant']=linked_all3['implant']

dxcode_pca_df = pca_feature_reduction(df_pt_dxcode, cols, 'dxcode', 'implant', 20)
dxcode_pca_df.head()

### 3.8. Create a "dept" feature matrix

In [ ]:
df_pt_dept, cols_dept_dict, cnt_dept_feature_sorted= create_pt_categorical_feature_matrix(linked_all3, 'dept')
df_pt_dept

In [ ]:
df_pt_dept, dept_vector_dict, dept_col_map = create_pt_vector(linked_all3, 'dept', 'tfidf', 1)
len(dept_vector_dict)
#df_pt_dx, dx_vector_dict, dx_col_map = create_pt_vector(linked_all3, 'diagnosis', 'tf', 20)

In [ ]:
dept_vector_dict

In [ ]:
df_pt_dept['implant']=linked_all3['implant']
corr = df_pt_dept.corr()

plt.figure(figsize=(10,8))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, vmax=1.0, vmin=-1, cmap=cmap)
plt.show();

### 3.9. Create a px feature matrix

df_pt_px, cols_px_dict, cnt_px_feature_sorted= create_pt_categorical_feature_matrix(linked_all3, 'purchase', 20)
cnt_px_feature_sorted

In [ ]:
df_pt_px, px_vector_dict, px_col_map= create_pt_vector(linked_all3, 'purchase', 'tf', 20)
len(px_vector_dict)

In [ ]:
df_pt_px['implant']=linked_all3['implant']
corr = df_pt_px.corr()

plt.figure(figsize=(10,8))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, vmax=1.0, vmin=-1, cmap=cmap)
plt.show();

In [ ]:
corr['implant']

In [ ]:
drop_px = corr[corr['implant'].abs()<0.005].index
{d: px_col_map[d] for d in drop_px}
df_pt_px.drop(drop_px, axis=1, inplace=True)

In [ ]:
cols = [c for c in df_pt_px.columns if 'purchase' in c]
df_pt_px['implant']=linked_all3['implant']

px_pca_df = pca_feature_reduction(df_pt_px, cols, 'px', 'implant', 40)
px_pca_df.head()

In [ ]:
def tsne_marker(df, cols, alias, label_col, n=2):
    
    x = df.loc[:, cols].values
    x = StandardScaler().fit_transform(x)
    
    y = df.loc[:, label_col].values
   

    #pca = PCA(n_components=m)
    tsne = TSNE(n_components=n)#, perplexity=20, n_iter=1000, learning_rate=200)
    #tsne_after_pca = Pipeline([('pca', pca), ('tsne', tsne)])
    dr = tsne.fit_transform(x)
   
    tsne_cols = [alias+'_tsne_'+str(k) for k in range(n)]
    tsne_df = pd.DataFrame(data=dr, columns=tsne_cols, index=df.index)
    final_tsne_df=pd.concat([tsne_df, df[[label_col]]], axis=1)
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    targets = [0, 1]
    colors = ['g', 'r']
    for target, color in zip(targets, colors):
        indicesToKeep = final_tsne_df[label_col] == target
        ax.scatter(final_tsne_df.loc[indicesToKeep, alias+'_tsne_0'], final_tsne_df.loc[indicesToKeep, alias+'_tsne_1'], c=color, s=20, alpha=0.3);
    ax.legend(targets);
    
    return tsne_df

cols = [c for c in df_pt_px.columns if 'purchase' in c]
px_tsne_df = tsne_marker(df_pt_px, cols, 'px', 'implant', 2)
px_tsne_df.head()
#x_train = tsne_marker(train_x, l, n_pca, n_tsne)

### 3.10 Create final feature matrix

In [ ]:

corr = linked_all3[['total_order', 'age', 'mean_order', 'visits', 'duration_days', 'implant']].corr()

plt.figure(figsize=(10,8))
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, vmax=1.0, vmin=-1, cmap=cmap, annot=True)
plt.show();

In [ ]:
print(df_pt_dx.shape, px_pca_df.shape, df_pt_dept.shape, dxcode_pca_df.shape)

In [ ]:
linked_df = pd.concat([linked_all3, 
                       df_pt_dx.drop(['implant'], axis=1), 
                       df_pt_dept.drop(['implant'], axis=1), 
                       px_pca_df.drop(['implant'], axis=1), 
                       dxcode_pca_df.drop(['implant'], axis=1)], axis=1)

linked_df.drop(['diagnosis', 'treatment_plan', 'purchase', 'order_type', 'date_list', 'mean_order', 'dept', 'dxcode'], axis=1, inplace=True)
linked_df.shape

In [ ]:
linked_df.head()

## 4. Train Model
### 4.1. Train-test data split

In [ ]:
X0 = linked_df.drop(axis = 1, columns = ['implant']).values
Y0 = linked_df['implant'].values
print(X0.shape, Y0.shape)
pd.Series(Y0).value_counts().plot.bar();

In [ ]:
def sampling(train_x0, train_y0, sampling):
    
    if sampling == 'under':
        nm = NearMiss(version=3)
        train_x, train_y = nm.fit_resample(train_x0, train_y0)
    elif sampling =='over':
        oversample = BorderlineSMOTE(random_state=10, kind="borderline-2")
        train_x, train_y = oversample.fit_resample(train_x0, train_y0.ravel())
    pd.Series(train_y).value_counts().plot.bar();
    
    return train_x, train_y

train_x0, test_x0, train_y0, test_y0 = train_test_split(X0, Y0, test_size = 0.3, random_state = 1)

train_x, train_y = sampling(train_x0, train_y0, 'under')
train_x = StandardScaler().fit_transform(train_x)

In [ ]:
test_x, test_y = sampling(test_x0, test_y0, 'under')
test_x = StandardScaler().fit_transform(test_x)

## Helper Fuctions: plot_lift_curve, plot_learning_curve

In [ ]:
def plot_lift_curve(y_val, y_pred, step=0.01):
    '''
    y_val: label of test data 
    y_pred: probability of prediction ons for such data
    step: resolution
    '''    
    aux_lift = pd.DataFrame()
    aux_lift['real'] = y_val
    aux_lift['predicted'] = y_pred
    # sorted by the predicted probability column:
    aux_lift.sort_values('predicted',ascending=False,inplace=True)
    
    x_val = np.arange(step, 1+step, step)
    # Ratio of positive label
    ratio_ones = aux_lift['real'].sum() / len(aux_lift)
    
    y_v = []
    for x in x_val:
        num_data = int(np.ceil(x*len(aux_lift))) # the # of data points based on ratio x 
        data_here = aux_lift.iloc[:num_data,:]  # a subset of sorted data
        ratio_ones_here = data_here['real'].sum()/len(data_here) # ratio of positive labels within this subset
        y_v.append(ratio_ones_here / ratio_ones)
           
   #Plot the figure
    fig, axis = plt.subplots()
    axis.plot(x_val, y_v, 'r-');
    axis.plot(x_val, np.ones(len(x_val)), 'g-');
    axis.set_xlabel('Proportion of sample');
    axis.set_ylabel('Lift');
    plt.show();

    return

def plot_learning_curve(model, train_x, train_y, test_x, test_y, n_splits=10):
    '''
    train_x, train_y: training data and training label
    test_x, test_y: training data and training label
    '''    
    X = np.concatenate((train_x, test_x), axis=0)
    Y = np.concatenate((test_y, train_y), axis=0)

    kfold = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=7) 
    
    LC = learning_curve(estimator=model, X=X, y=Y, cv=kfold, train_sizes=np.linspace(0.10, 1.00, 20))

    #plt.figure(figsize =(12, 10))
    plt.plot(LC[0],np.nanmean(LC[1], axis=1),c='blue');
    plt.plot(LC[0],np.nanmean(LC[2], axis=1), c='green');
    plt.show();

    return



### 4.2. Baseline model:

In [ ]:
rfc = RandomForestClassifier(oob_score = True).fit(train_x, train_y)
prob_rfc = rfc.predict_proba(test_x)
class_rfc = rfc.predict(test_x)

print(classification_report(test_y, class_rfc))
print('AUC: ', roc_auc_score(test_y, prob_rfc[:,1]))

In [ ]:
y_score = prob_rfc[:, 1]

fpr, tpr, _ = roc_curve(test_y, y_score, pos_label=rfc.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
prec, recall, _ = precision_recall_curve(test_y, y_score, pos_label=rfc.classes_[1])
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

In [ ]:
from sklearn.model_selection import learning_curve

X1 = np.concatenate((train_x, test_x), axis=0)
Y1 = np.concatenate((test_y, train_y), axis=0)

kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=7) 

LC = learning_curve(estimator=rfc, X=X1, y=Y1, cv=kfold, train_sizes=np.linspace(0.10, 1.00, 10))

#plt.figure(figsize =(12, 10))
plt.plot(LC[0],np.nanmean(LC[1], axis=1), c='blue');
plt.plot(LC[0],np.nanmean(LC[2], axis=1), c='green');
plt.show();

### 4.3. Recursivce Feature Elimination (RFECV) in feature selecting to avoid over-fitting

In [ ]:
rfecv = RFECV(estimator=rfc, step=3, cv=StratifiedKFold(10), scoring='recall')
rfecv.fit(train_x, train_y)

print('The optimal number of features: {}'.format(rfecv.n_features_))
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_,)
plt.show();

In [ ]:
feature_list = linked_df.drop(axis = 1, columns = ['implant']).columns
col_idx = {i: v for i, v in enumerate(feature_list)}

In [ ]:
feature_list = linked_df.drop(axis = 1, columns = ['implant']).columns

selected_feature_list=[feature_list[i] for i, v in enumerate(rfecv.support_ ) if v == True]

selected_features = dict(zip(selected_feature_list, rfecv.estimator_.feature_importances_))

df_selected_features = pd.DataFrame().from_dict(selected_features, orient='index', columns=['importance']).sort_values(by=['importance'], ascending=False)
df_selected_features.head(10)

df_selected_features.plot.barh(figsize=(10, 8));

In [ ]:
len(selected_feature_list)

### 4.4. Re-train model with selected featuers

In [ ]:
def sampling(train_x0, train_y0, sampling):
    
    if sampling == 'under':
        nm = NearMiss(version=3)
        train_x, train_y = nm.fit_resample(train_x0, train_y0)
    elif sampling =='over':
        oversample = BorderlineSMOTE(random_state=10, kind="borderline-2")
        train_x, train_y = oversample.fit_resample(train_x0, train_y0.ravel())
    pd.Series(train_y).value_counts().plot.bar();
    
    return train_x, train_y

X1 = linked_df.drop(axis = 1, columns = ['implant'])[selected_feature_list].values
Y1 = linked_df['implant'].values

train_x0, test_x0, train_y0, test_y0 = train_test_split(X1, Y1, test_size = 0.3, random_state = 1)

train_x, train_y = sampling(train_x0, train_y0, 'over')
train_x = StandardScaler().fit_transform(train_x)

In [ ]:
test_x, test_y = sampling(test_x0, test_y0, 'over')
test_x = StandardScaler().fit_transform(test_x)

In [ ]:
rfc2 = RandomForestClassifier(oob_score = True).fit(train_x, train_y)
prob_out = rfc.predict_proba(test_x)
class_out = rfc.predict(test_x)

print(classification_report(test_y, class_out))
print('AUC: ', roc_auc_score(test_y, prob_out[:,1]))

In [ ]:
# get probablity output of the positive 
print(rfc.classes_)
y_score = prob_out[:, 1]

# ROC curve
fpr, tpr, _ = roc_curve(test_y, y_score, pos_label=rfc.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

# precision-recall curve
prec, recall, _ = precision_recall_curve(test_y, y_score, pos_label=rfc.classes_[1])
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

# lift-curve
plot_lift_curve(test_y, y_score, 0.02)

In [ ]:
# learning curve
plot_learning_curve(rfc2, train_x, train_y, test_x, test_y, 5)

In [ ]:
print(rfc2.get_params())

In [ ]:
max_depth = np.arange(10, 100, 10)
min_samples_leaf = np.arange(1, 4, 2)
bootstrap=[True, False]

param_grid = dict(min_samples_leaf=min_samples_leaf, max_depth=max_depth, bootstrap=bootstrap)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

grid_search = GridSearchCV(rfc2 ,param_grid=param_grid, scoring='recall', n_jobs=-1, cv=kfold)
grid_result = gridsearch.fit(train_x, train_y) 
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, gridsearch.best_params_))

best_grid = gridsearch.best_estimator_

prob_out = best_grid.predict_proba(test_x)
class_out = best_grid.predict(test_x)

print(classification_report(test_y, class_out))
print('AUC: ', roc_auc_score(test_y, prob_out[:,1]))


In [ ]:
# get probablity output of the positive 
#print(rfc.classes_)
y_score = prob_out[:, 1]

# ROC curve
fpr, tpr, _ = roc_curve(test_y, y_score, pos_label=rfc.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

# precision-recall curve
prec, recall, _ = precision_recall_curve(test_y, y_score, pos_label=rfc.classes_[1])
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

# lift-curve
plot_lift_curve(test_y, y_score, 0.02)

In [ ]:
# learning curve
plot_learning_curve(best_grid, train_x, train_y, test_x, test_y)

In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance

XGB = XGBClassifier(objective = "binary:logistic", tree_method='auto')
XGB.fit(train_x, train_y, eval_set = [(test_x, test_y)], verbose = 0)

prob_out = XGB.predict_proba(test_x)
class_out = XGB.predict(test_x)

print(classification_report(test_y, class_out))
roc_auc_score(test_y, prob_out[:,1])

y_score = prob_out[:, 1]
# ROC curve
fpr, tpr, _ = roc_curve(test_y, y_score, pos_label=XGB.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

# precision-recall curve
prec, recall, _ = precision_recall_curve(test_y, y_score, pos_label=XGB.classes_[1])
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

In [ ]:
def f_importance_plot(model, metric = "gain"):

    fig,ax = plt.subplots(figsize=(12, 10))
    plot_importance(model, height=0.8, ax=ax, max_num_features=100, importance_type = metric)

    id_l = [int(re.findall(r"\d+",str(i))[2]) for i in plt.yticks()[1]]
    id_l.reverse()
    f_name = [selected_feature_list[i] for i in id_l]
    plt.yticks(range(len(f_name)),f_name,fontsize = 10)
    
    plt.show()

f_importance_plot(XGB, metric ='gain')